In [10]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import random

# --- 1. Konfigurasi Game ---
SYMBOLS = ['⭐', '🍒', '🍎', '🍋', '🍇', '🍉', '🔔', '💎'] * 2
random.shuffle(SYMBOLS)

cards_data = []
buttons = []
flipped_indices = []
matches_found = 0
can_click = True

# Kontainer untuk pesan status dan tombol Lanjutkan (akan selalu ditampilkan di tempat yang sama)
status_output = widgets.Output()
control_box = widgets.VBox([]) # <-- Kontainer baru untuk tombol "Lanjutkan"

# --- 2. Fungsi Logika Game ---

def reset_game_state():
    """Mengaktifkan kembali klik dan membersihkan control box."""
    global can_click
    can_click = True

    # Kosongkan control box (hapus tombol 'Lanjutkan')
    control_box.children = []

    with status_output:
        clear_output(wait=True)
        print("Memory Game: Silakan klik kartu berikutnya.")

def unflip_and_continue(b):
    """Fungsi yang dipanggil oleh tombol Lanjutkan."""
    global flipped_indices

    # Tutup kembali kartu yang tidak cocok
    idx1, idx2 = flipped_indices

    buttons[idx1].description = '?'
    buttons[idx2].description = '?'
    buttons[idx1].style.button_color = 'lightblue'
    buttons[idx2].style.button_color = 'lightblue'
    cards_data[idx1]['is_flipped'] = False
    cards_data[idx2]['is_flipped'] = False

    flipped_indices = []

    # Lanjutkan permainan
    reset_game_state()

def handle_click(b):
    """Fungsi yang dipanggil saat tombol (kartu) diklik."""
    global flipped_indices, matches_found, can_click

    if not can_click:
        return

    clicked_index = buttons.index(b)

    # Cek: Sudah cocok atau sudah terbuka?
    if cards_data[clicked_index]['matched'] or clicked_index in flipped_indices:
        return

    # 1. Buka Kartu
    cards_data[clicked_index]['is_flipped'] = True
    b.description = cards_data[clicked_index]['symbol']
    b.style.button_color = 'yellow'
    flipped_indices.append(clicked_index)

    with status_output:
        clear_output(wait=True)

    # 2. Cek Pasangan
    if len(flipped_indices) == 2:
        can_click = False # Nonaktifkan klik kartu saat mengecek

        idx1, idx2 = flipped_indices
        symbol1 = cards_data[idx1]['symbol']
        symbol2 = cards_data[idx2]['symbol']

        if symbol1 == symbol2:
            # Pasangan Cocok!
            matches_found += 1
            cards_data[idx1]['matched'] = True
            cards_data[idx2]['matched'] = True
            buttons[idx1].style.button_color = 'lightgreen'
            buttons[idx2].style.button_color = 'lightgreen'
            buttons[idx1].disabled = True
            buttons[idx2].disabled = True

            flipped_indices = []
            can_click = True # Aktifkan klik kembali

            if matches_found == len(SYMBOLS) / 2:
                with status_output:
                    print("🎉 SELAMAT! Kamu berhasil menemukan semua pasangan!")

        else:
            # Tidak Cocok. Tampilkan tombol Lanjutkan di control_box
            continue_button = widgets.Button(
                description='Lanjutkan',
                button_style='danger'
            )
            continue_button.on_click(unflip_and_continue)

            with status_output:
                print("❌ Tidak Cocok! Lihat baik-baik. Klik 'Lanjutkan' untuk menutup kartu.")

            # Hapus item lama dan tambahkan tombol baru ke control_box
            control_box.children = [continue_button]

# --- 3. Inisialisasi dan Tampilkan Antarmuka ---

for i, symbol in enumerate(SYMBOLS):
    cards_data.append({
        'symbol': symbol,
        'is_flipped': False,
        'matched': False
    })

    button = widgets.Button(
        description='?',
        layout=widgets.Layout(width='80px', height='80px'),
        style=widgets.ButtonStyle(button_color='lightblue')
    )
    button.on_click(handle_click)
    buttons.append(button)

game_grid = widgets.GridBox(
    buttons,
    layout=widgets.Layout(
        width='auto',
        grid_template_columns='repeat(4, 1fr)',
        grid_gap='10px'
    )
)

with status_output:
    print("Memory Game: Klik 2 kartu untuk memulai.")

# Tampilkan semua kontainer utama sekali saja
display(status_output, game_grid, control_box)

Output()

GridBox(children=(Button(description='?', layout=Layout(height='80px', width='80px'), style=ButtonStyle(button…

VBox()

In [23]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import random

# --- 1. Konfigurasi Game ---
CHOICES = ['batu', 'gunting', 'kertas']

EMOJI_MAP = {
    'batu': '👊',
    'gunting': '✂️',
    'kertas': '🖐️',
    'default': '❓'
}
EMOJI_BUTTON_MAP = {
    'batu': '👊 Batu',
    'gunting': '✂️ Gunting',
    'kertas': '🖐️ Kertas'
}

BOX_HEIGHT = '100px'
BOX_WIDTH = '120px'

# Kontainer untuk menampilkan hasil pertandingan (output_result_area)
output_result_area = widgets.Output(
    layout=widgets.Layout(
        width='150px',
        align_items='center',
        overflow='hidden'  # <--- PERUBAHAN UTAMA DI SINI
    )
)

# --- WIDGET UNTUK TAMPILAN SIMBOL DI BAWAH ---
user_symbol_output = widgets.Output()
computer_symbol_output = widgets.Output()

# --- 2. Fungsi Logika Permainan ---

def get_computer_choice():
    """Memilih gerakan komputer secara acak."""
    return random.choice(CHOICES)

def update_symbol_display(output_widget, choice, is_user=False):
    """Memperbarui tampilan simbol pemain atau komputer."""
    border_color = '#28a745' if is_user else '#dc3545'

    style = f"border: 3px solid {border_color}; padding: 10px; font-weight: bold; background-color: #f0f0f0; border-radius: 5px; text-align: center; color: #333; height: {BOX_HEIGHT}; width: {BOX_WIDTH}; display: flex; flex-direction: column; justify-content: center; align-items: center;"

    with output_widget:
        clear_output(wait=True)

        emoji = EMOJI_MAP.get(choice, EMOJI_MAP['default'])
        choice_text = choice.upper() if choice else 'Pilih!'

        display(widgets.HTML(f"""
            <div style='{style}'>
                <span style='font-size: 48px;'>{emoji}</span>
                <span style='font-size: 16px; margin-top: 5px;'>{choice_text}</span>
            </div>
        """))


def determine_winner(user_choice, computer_choice):
    """Menentukan pemenang dan memperbarui semua hasil."""

    # 1. Update Tampilan Simbol Berdampingan
    update_symbol_display(user_symbol_output, user_choice, is_user=True)
    update_symbol_display(computer_symbol_output, computer_choice, is_user=False)

    # 2. Tentukan Hasil
    if user_choice == computer_choice:
        result = "Seri"
        color = "#ffc107" # Kuning
    elif (user_choice == 'batu' and computer_choice == 'gunting') or \
         (user_choice == 'gunting' and computer_choice == 'kertas') or \
         (user_choice == 'kertas' and computer_choice == 'batu'):
        result = "MENANG"
        color = "#28a745" # Hijau
    else:
        result = "KALAH"
        color = "#dc3545" # Merah

    # 3. Tampilkan Hasil Besar di Output Area
    with output_result_area:
        clear_output(wait=True)
        # Style HTML untuk teks besar dan terpusat (tambahkan overflow: hidden)
        html_style = f"text-align: center; font-size: 32px; font-weight: bold; margin: 0; padding: 0; color: {color}; overflow: hidden;" # <--- PERBAIKAN DI SINI
        display(widgets.HTML(f"<div style='{html_style}'>{result.upper()}!</div>"))


def handle_click(b):
    """Fungsi yang dipanggil saat salah satu tombol diklik."""
    user_choice = b.description.split(' ')[1].lower()
    computer_choice = get_computer_choice()

    determine_winner(user_choice, computer_choice)

# --- 3. Inisialisasi Tombol dan Tampilkan Antarmuka ---

buttons_list = []
styles_map = {
    'batu': 'info',
    'gunting': 'warning',
    'kertas': 'success'
}

for choice in CHOICES:
    button_label = EMOJI_BUTTON_MAP[choice]
    button_style = styles_map[choice]

    button = widgets.Button(
        description=button_label,
        button_style=button_style,
        layout=widgets.Layout(width=BOX_WIDTH, height=BOX_HEIGHT)
    )
    button.on_click(handle_click)
    buttons_list.append(button)

user_choices_box = widgets.HBox(buttons_list)

# --- BAGIAN BAWAH: Tampilan Simbol Pertandingan ---
vs_label = widgets.Label(
    value="VS",
    style={'font_weight': 'bold', 'font_size': '30px'},
    layout=widgets.Layout(margin='0 10px', align_self='center')
)

bottom_section = widgets.VBox([
    widgets.Label("Arena Pertandingan:", style={'font_weight': 'bold', 'text_align': 'center', 'font_size': '18px'}),
    widgets.HBox([
        widgets.VBox([
            widgets.Label("KAMU"),
            user_symbol_output
        ], layout=widgets.Layout(align_items='center')),
        vs_label,
        widgets.VBox([
            widgets.Label("KOMPUTER"),
            computer_symbol_output
        ], layout=widgets.Layout(align_items='center')),
    ], layout=widgets.Layout(justify_content='center', margin='10px 0'))
])


# --- BAGIAN ATAS: Pilihan User dan Hasil ---
user_controls_stack = widgets.VBox([
    widgets.Label("Pilihanmu:", style={'font_weight': 'bold'}),
    user_choices_box,
    widgets.Label("Hasil Pertandingan:", style={'font_weight': 'bold', 'text_align': 'center', 'margin_top': '10px'}),
    output_result_area
], layout=widgets.Layout(align_items='center'))


# Gabungkan semua bagian dalam satu VBox utama
main_game_display = widgets.VBox([
    widgets.HBox([user_controls_stack], layout=widgets.Layout(justify_content='center')),
    widgets.HBox([], layout=widgets.Layout(height='2px', background_color='#ccc', width='100%', margin='15px 0')),
    bottom_section
])


# Tampilkan semua komponen game
print("--- Batu, Gunting, Kertas ---")
display(main_game_display)

# Inisialisasi tampilan simbol di bawah
update_symbol_display(user_symbol_output, None, is_user=True)
update_symbol_display(computer_symbol_output, None, is_user=False)

--- Batu, Gunting, Kertas ---


In [24]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import random

# --- 1. Konfigurasi Global dan Terjemahan ---

# Pilihan inti tetap dalam bahasa Inggris untuk logika
CHOICES = ['rock', 'scissors', 'paper']
CURRENT_LANGUAGE = 'id' # Default bahasa Indonesia

# Kamus terjemahan untuk semua teks dalam game
translations = {
    'id': {
        'title': "Batu, Gunting, Kertas",
        'choose_lang': "Pilih Bahasa:",
        'lang_id': "🇮🇩 Bahasa Indonesia",
        'lang_en': "🇬🇧 English",
        'choice_label': "Pilihanmu:",
        'result_label': "Hasil Pertandingan:",
        'arena_label': "Arena Pertandingan:",
        'player_label': "KAMU",
        'comp_label': "KOMPUTER",
        'vs': "VS",
        'win': "MENANG",
        'lose': "KALAH",
        'draw': "SERI",
        'rock': "Batu",
        'scissors': "Gunting",
        'paper': "Kertas",
        'choose_prompt': "Pilih!",
        'comp_prompt': "PIL. KOMP."
    },
    'en': {
        'title': "Rock, Paper, Scissors",
        'choose_lang': "Choose Language:",
        'lang_id': "🇮🇩 Bahasa Indonesia",
        'lang_en': "🇬🇧 English",
        'choice_label': "Your Choice:",
        'result_label': "Match Result:",
        'arena_label': "Battle Arena:",
        'player_label': "YOU",
        'comp_label': "COMPUTER",
        'vs': "VS",
        'win': "WIN",
        'lose': "LOSE",
        'draw': "DRAW",
        'rock': "Rock",
        'scissors': "Scissors",
        'paper': "Paper",
        'choose_prompt': "Choose!",
        'comp_prompt': "COMP. CHOICE"
    }
}

# Mapping Emoji
EMOJI_MAP = {
    'rock': '👊',
    'scissors': '✂️',
    'paper': '🖐️',
    'default': '❓'
}

# Variabel Layout
BOX_HEIGHT = '100px'
BOX_WIDTH = '120px'

# Kontainer (perlu diinisialisasi sebelum digunakan)
output_result_area = widgets.Output(layout=widgets.Layout(width='150px', align_items='center'))
user_symbol_output = widgets.Output()
computer_symbol_output = widgets.Output()
user_choices_box = widgets.HBox([]) # Placeholder untuk tombol
main_game_display = widgets.VBox([]) # Placeholder untuk tampilan utama

# --- 2. Fungsi Utama Game ---

def get_text(key):
    """Fungsi pembantu untuk mengambil terjemahan."""
    return translations[CURRENT_LANGUAGE].get(key, key)

def build_game_ui():
    """Membangun antarmuka game (tombol, label, dan tata letak)."""
    global user_choices_box, main_game_display

    # --- Hapus tampilan lama ---
    if hasattr(main_game_display, 'close'):
        main_game_display.close()

    # --- 2.1: Buat Tombol Pilihan User (dengan teks bahasa yang benar) ---
    buttons_list = []
    styles_map = {'rock': 'info', 'scissors': 'warning', 'paper': 'success'}

    for choice_en in CHOICES:
        # Teks tombol adalah Emoji + Terjemahan
        button_label = f"{EMOJI_MAP[choice_en]} {get_text(choice_en)}"

        button = widgets.Button(
            description=button_label,
            button_style=styles_map[choice_en],
            layout=widgets.Layout(width=BOX_WIDTH, height=BOX_HEIGHT)
        )
        button.on_click(handle_click)
        buttons_list.append(button)

    user_choices_box = widgets.HBox(buttons_list)

    # --- 2.2: Bagian Bawah: Tampilan Simbol Pertandingan ---
    vs_label = widgets.Label(
        value=get_text('vs'),
        style={'font_weight': 'bold', 'font_size': '30px'},
        layout=widgets.Layout(margin='0 10px', align_self='center')
    )

    bottom_section = widgets.VBox([
        widgets.Label(get_text('arena_label'), style={'font_weight': 'bold', 'text_align': 'center', 'font_size': '18px'}),
        widgets.HBox([
            widgets.VBox([
                widgets.Label(get_text('player_label')),
                user_symbol_output
            ], layout=widgets.Layout(align_items='center')),
            vs_label,
            widgets.VBox([
                widgets.Label(get_text('comp_label')),
                computer_symbol_output
            ], layout=widgets.Layout(align_items='center')),
        ], layout=widgets.Layout(justify_content='center', margin='10px 0'))
    ])

    # --- 2.3: Bagian Atas: Pilihan User dan Hasil ---
    user_controls_stack = widgets.VBox([
        widgets.Label(get_text('choice_label'), style={'font_weight': 'bold'}),
        user_choices_box,
        widgets.Label(get_text('result_label'), style={'font_weight': 'bold', 'text_align': 'center', 'margin_top': '10px'}),
        output_result_area
    ], layout=widgets.Layout(align_items='center'))

    # --- 2.4: Gabungkan semua bagian ---
    main_game_display = widgets.VBox([
        widgets.HBox([user_controls_stack], layout=widgets.Layout(justify_content='center')),
        widgets.HBox([], layout=widgets.Layout(height='2px', background_color='#ccc', width='100%', margin='15px 0')),
        bottom_section
    ])

    # Tampilkan UI baru
    display(main_game_display)

    # Inisialisasi tampilan simbol di bawah
    update_symbol_display(user_symbol_output, None, is_user=True)
    update_symbol_display(computer_symbol_output, None, is_user=False)

# --- Fungsi Update Tampilan dan Logika ---

def update_symbol_display(output_widget, choice_en, is_user=False):
    """Memperbarui tampilan simbol pemain atau komputer."""
    border_color = '#28a745' if is_user else '#dc3545'
    style = f"border: 3px solid {border_color}; padding: 10px; font-weight: bold; background-color: #f0f0f0; border-radius: 5px; text-align: center; color: #333; height: {BOX_HEIGHT}; width: {BOX_WIDTH}; display: flex; flex-direction: column; justify-content: center; align-items: center;"

    with output_widget:
        clear_output(wait=True)

        emoji = EMOJI_MAP.get(choice_en, EMOJI_MAP['default'])

        # Teks di bawah simbol menggunakan terjemahan
        if choice_en:
            choice_text = get_text(choice_en).upper()
        else:
            choice_text = get_text('choose_prompt').upper() if is_user else get_text('comp_prompt').upper()

        display(widgets.HTML(f"""
            <div style='{style}'>
                <span style='font-size: 48px;'>{emoji}</span>
                <span style='font-size: 16px; margin-top: 5px;'>{choice_text}</span>
            </div>
        """))


def determine_winner(user_choice_en, computer_choice_en):
    """Menentukan pemenang dan memperbarui semua hasil."""

    # 1. Update Tampilan Simbol Berdampingan
    update_symbol_display(user_symbol_output, user_choice_en, is_user=True)
    update_symbol_display(computer_symbol_output, computer_choice_en, is_user=False)

    # 2. Tentukan Hasil (Logika menggunakan EN)
    if user_choice_en == computer_choice_en:
        result_key = 'draw'
        color = "#ffc107"
    elif (user_choice_en == 'rock' and computer_choice_en == 'scissors') or \
         (user_choice_en == 'scissors' and computer_choice_en == 'paper') or \
         (user_choice_en == 'paper' and computer_choice_en == 'rock'):
        result_key = 'win'
        color = "#28a745"
    else:
        result_key = 'lose'
        color = "#dc3545"

    # 3. Tampilkan Hasil Besar di Output Area (menggunakan terjemahan)
    with output_result_area:
        clear_output(wait=True)
        html_style = f"text-align: center; font-size: 32px; font-weight: bold; margin: 0; padding: 0; color: {color}; overflow: hidden;"
        result_text = get_text(result_key)
        display(widgets.HTML(f"<div style='{html_style}'>{result_text.upper()}!</div>"))


def handle_click(b):
    """Fungsi yang dipanggil saat salah satu tombol diklik."""
    # Ambil pilihan dalam format kunci bahasa Inggris ('rock', 'scissors', 'paper')
    # b.description = '👊 Batu' atau '👊 Rock'
    # Ambil kata kedua, cari kuncinya dari kamus terbalik

    text_choice = b.description.split(' ')[1].lower()

    # Cari kunci EN berdasarkan teks yang diklik
    user_choice_en = next(
        (key for key, value in translations[CURRENT_LANGUAGE].items()
         if value.lower() == text_choice), None
    )

    if user_choice_en:
        computer_choice_en = get_computer_choice()
        determine_winner(user_choice_en, computer_choice_en)

# --- 3. Pilihan Bahasa dan Inisialisasi ---

lang_output = widgets.Output()

def set_language(b):
    """Mengatur bahasa dan membangun ulang UI game."""
    global CURRENT_LANGUAGE

    if b.description == translations['id']['lang_id']:
        CURRENT_LANGUAGE = 'id'
    elif b.description == translations['en']['lang_en']:
        CURRENT_LANGUAGE = 'en'

    with lang_output:
        clear_output(wait=True)
        print(f"Bahasa diatur ke: {translations[CURRENT_LANGUAGE]['title']}")

    build_game_ui() # Bangun UI game dalam bahasa baru

# Tombol Pilihan Bahasa
id_button = widgets.Button(description=translations['id']['lang_id'], button_style='primary')
en_button = widgets.Button(description=translations['en']['lang_en'], button_style='primary')

id_button.on_click(set_language)
en_button.on_click(set_language)

# Tampilan Pilihan Bahasa
lang_selector = widgets.VBox([
    widgets.Label(translations['en']['choose_lang']),
    widgets.HBox([id_button, en_button])
])

print(f"--- {translations['en']['title']} ---")
display(lang_selector, lang_output)

--- Rock, Paper, Scissors ---


Output()